In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from scipy.stats import skew
import numpy as np
import re
import os
from datetime import timedelta

### Create one dataframe containing all events for each resolution

In [44]:
# all_events_5mins =[]
# all_events_10mins =[]
# all_events_30mins =[]
# all_events_60mins =[]

# total_events = 0
# print(len(os.listdir("../DanishRainData_Outputs/5mins/")))

# for num, file in enumerate(np.sort(os.listdir("../DanishRainData_Outputs/5mins/"))):
#     if file != "All_events_571820_precip_minute.csv":
#         # Get 5 mintue data
#         df_5m = pd.read_csv(f"../DanishRainData_Outputs/5mins/{file}")
#         df_5m['event_num']=df_5m.index
#         # Get rid of the trailing strings sometimes on the gauge_num
#         df_5m['gauge_num'] = df_5m['gauge_num'].apply(
#             lambda x: int(x.split('_')[0]) if isinstance(x, str) and x.split('_')[0].isdigit() else x)

#         # Get other resolution data
#         df_10m = pd.read_csv(f"../DanishRainData_Outputs/{10}mins/{file}")
#         df_30m = pd.read_csv(f"../DanishRainData_Outputs/{30}mins/{file}")
#         df_60m = pd.read_csv(f"../DanishRainData_Outputs/{60}mins/{file}")

#        # Find common event numbers in all four
#         common_event_nums = (
#             set(df_5m['event_num']) &
#             set(df_10m['event_num']) &
#             set(df_30m['event_num']) &
#             set(df_60m['event_num']))

#         # Filter all DataFrames to only include common events
#         df_5m = df_5m[df_5m['event_num'].isin(common_event_nums)].reset_index(drop=True)
#         df_10m = df_10m[df_10m['event_num'].isin(common_event_nums)].reset_index(drop=True)
#         df_30m = df_30m[df_30m['event_num'].isin(common_event_nums)].reset_index(drop=True)
#         df_60m = df_60m[df_60m['event_num'].isin(common_event_nums)].reset_index(drop=True)

#         total_events = total_events + len(common_event_nums)
#         print(f"{num} : {file}: {len(common_event_nums)} events retained across all resolutions, new total: {total_events}")    

#         # Collect DataFrames into a dictionary for easy iteration
#         dfs = {'5m': df_5m,
#             '10m': df_10m,
#             '30m': df_30m,
#             '60m': df_60m}

#         # Check the start and end times align
#         for i in range(len(df_5m)):
#             reference_start = pd.to_datetime(df_5m.iloc[i]['start_time'])
#             reference_end = pd.to_datetime(df_5m.iloc[i]['end_time'])
#             # print(reference_start,reference_end)
#             for res, df in dfs.items():
#                 start = pd.to_datetime(df.iloc[i]['start_time'])
#                 end = pd.to_datetime(df.iloc[i]['end_time'])
#                 #print(res, start, end)

#                 delta_start = abs(start - reference_start)
#                 delta_end = abs(end - reference_end)

#                 if delta_start > timedelta(minutes=60):
#                     print(f"⚠️ START mismatch in {res} at index {i}")
#                     print(f"   {res} start: {start}, 5m start: {reference_start}, Δ: {delta_start}")

#                 if delta_end > timedelta(minutes=60):
#                     print(f"⚠️ END mismatch in {res} at index {i}")
#                     print(f"   {res} end: {end}, 5m end: {reference_end}, Δ: {delta_end}")

#         all_events_5mins.append(df_5m)
#         all_events_10mins.append(df_10m)             
#         all_events_30mins.append(df_30m)
#         all_events_60mins.append(df_60m)            

# all_events_df_5mins = pd.concat(all_events_5mins)   
# all_events_df_10mins = pd.concat(all_events_10mins)   
# all_events_df_30mins = pd.concat(all_events_30mins)
# all_events_df_60mins = pd.concat(all_events_60mins)   

# for df in [all_events_df_5mins, all_events_df_10mins, all_events_df_30mins, all_events_df_60mins]:
#     df.reset_index(drop=True, inplace=True)
    
# print(len(all_events_df_5mins), len(all_events_df_10mins), len(all_events_df_30mins), len(all_events_df_60mins))

346
0 : All_events_500520_precip_minute.csv: 986 events retained across all resolutions, new total: 986
1 : All_events_500920_precip_minute.csv: 870 events retained across all resolutions, new total: 1856
2 : All_events_5012_svk_precip_minute.csv: 242 events retained across all resolutions, new total: 2098
3 : All_events_501520_precip_minute.csv: 843 events retained across all resolutions, new total: 2941
4 : All_events_5025_svk_precip_minute.csv: 2187 events retained across all resolutions, new total: 5128
5 : All_events_5027_svk_precip_minute.csv: 2306 events retained across all resolutions, new total: 7434
6 : All_events_503120_precip_minute.csv: 917 events retained across all resolutions, new total: 8351
7 : All_events_5032_svk_precip_minute.csv: 346 events retained across all resolutions, new total: 8697
8 : All_events_503520_precip_minute.csv: 907 events retained across all resolutions, new total: 9604
9 : All_events_504020_precip_minute.csv: 208 events retained across all resolu

79 : All_events_5243_svk_precip_minute.csv: 2164 events retained across all resolutions, new total: 95241
80 : All_events_5245_svk_precip_minute.csv: 1140 events retained across all resolutions, new total: 96381
81 : All_events_5247_svk_precip_minute.csv: 1126 events retained across all resolutions, new total: 97507
82 : All_events_5248_svk_precip_minute.csv: 1148 events retained across all resolutions, new total: 98655
83 : All_events_5251_svk_precip_minute.csv: 3340 events retained across all resolutions, new total: 101995
84 : All_events_5252_svk_precip_minute.csv: 1136 events retained across all resolutions, new total: 103131
85 : All_events_5254_svk_precip_minute.csv: 851 events retained across all resolutions, new total: 103982
86 : All_events_5255_svk_precip_minute.csv: 1135 events retained across all resolutions, new total: 105117
87 : All_events_5257_svk_precip_minute.csv: 1149 events retained across all resolutions, new total: 106266
88 : All_events_5260_svk_precip_minute.csv

157 : All_events_5485_svk_precip_minute.csv: 1368 events retained across all resolutions, new total: 183622
158 : All_events_5490_svk_precip_minute.csv: 1907 events retained across all resolutions, new total: 185529
159 : All_events_5495_svk_precip_minute.csv: 654 events retained across all resolutions, new total: 186183
160 : All_events_549920_precip_minute.csv: 929 events retained across all resolutions, new total: 187112
161 : All_events_5501_svk_precip_minute.csv: 21 events retained across all resolutions, new total: 187133
162 : All_events_5502_svk_precip_minute.csv: 20 events retained across all resolutions, new total: 187153
163 : All_events_5503_svk_precip_minute.csv: 23 events retained across all resolutions, new total: 187176
164 : All_events_5504_svk_precip_minute.csv: 20 events retained across all resolutions, new total: 187196
165 : All_events_550520_precip_minute.csv: 720 events retained across all resolutions, new total: 187916
166 : All_events_5509_svk_precip_minute.csv

234 : All_events_5675_svk_precip_minute.csv: 1283 events retained across all resolutions, new total: 248161
235 : All_events_5680_svk_precip_minute.csv: 1297 events retained across all resolutions, new total: 249458
236 : All_events_5685_svk_precip_minute.csv: 1234 events retained across all resolutions, new total: 250692
237 : All_events_5690_svk_precip_minute.csv: 1244 events retained across all resolutions, new total: 251936
238 : All_events_5694_svk_precip_minute.csv: 3002 events retained across all resolutions, new total: 254938
239 : All_events_5697_svk_precip_minute.csv: 927 events retained across all resolutions, new total: 255865
240 : All_events_5698_svk_precip_minute.csv: 497 events retained across all resolutions, new total: 256362
241 : All_events_5699_svk_precip_minute.csv: 1149 events retained across all resolutions, new total: 257511
242 : All_events_5703_svk_precip_minute.csv: 915 events retained across all resolutions, new total: 258426
243 : All_events_5705_svk_preci

311 : All_events_599420_precip_minute.csv: 846 events retained across all resolutions, new total: 351013
312 : All_events_601900_precip_minute.csv: 533 events retained across all resolutions, new total: 351546
313 : All_events_603000_precip_minute.csv: 660 events retained across all resolutions, new total: 352206
314 : All_events_603100_precip_minute.csv: 760 events retained across all resolutions, new total: 352966
315 : All_events_604100_precip_minute.csv: 151 events retained across all resolutions, new total: 353117
316 : All_events_605101_precip_minute.csv: 396 events retained across all resolutions, new total: 353513
317 : All_events_605200_precip_minute.csv: 740 events retained across all resolutions, new total: 354253
318 : All_events_605800_precip_minute.csv: 80 events retained across all resolutions, new total: 354333
319 : All_events_606000_precip_minute.csv: 911 events retained across all resolutions, new total: 355244
320 : All_events_606500_precip_minute.csv: 836 events re

In [45]:
# all_events_df_5mins.to_csv('all_events_df_5mins.csv', index=False)
# all_events_df_10mins.to_csv('all_events_df_10mins.csv', index=False)
# all_events_df_30mins.to_csv('all_events_df_30mins.csv', index=False)
# all_events_df_60mins.to_csv('all_events_df_60mins.csv', index=False)

### Scale and transform

In [5]:
all_events_df_5mins = pd.read_csv('Data/all_events_df_5mins.csv')
all_events_df_10mins = pd.read_csv('Data/all_events_df_10mins.csv')
all_events_df_30mins = pd.read_csv('Data/all_events_df_30mins.csv')
all_events_df_60mins = pd.read_csv('Data/all_events_df_60mins.csv')

In [16]:
all_events_df_5mins = all_events_df_5mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})
all_events_df_10mins = all_events_df_10mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})
all_events_df_30mins = all_events_df_30mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})
all_events_df_60mins = all_events_df_60mins.rename(columns={'lorentz_asymetry': 'lorenz_asymmetry'})

all_events_df_5mins = all_events_df_5mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})
all_events_df_10mins = all_events_df_10mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})
all_events_df_30mins = all_events_df_30mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})
all_events_df_60mins = all_events_df_60mins.rename(columns={'lorentz_asymetry_DMC_10': 'lorenz_asymmetry_DMC_10'})

### Remove rows with unrealistically high precip

In [17]:
bad_indices = all_events_df_5mins.sort_values(by="total_precip", ascending=False)[:15].index

# Drop rows at these indices from all dataframes
all_events_df_5mins = all_events_df_5mins.drop(index=bad_indices)
all_events_df_10mins = all_events_df_10mins.drop(index=bad_indices)
all_events_df_30mins = all_events_df_30mins.drop(index=bad_indices)
all_events_df_60mins = all_events_df_60mins.drop(index=bad_indices)

for df in [all_events_df_5mins, all_events_df_10mins, all_events_df_30mins, all_events_df_60mins]:
    df.reset_index(drop=True, inplace=True)
    
print(len(all_events_df_5mins), len(all_events_df_10mins), len(all_events_df_30mins), len(all_events_df_60mins))

364232 364232 364232 364232


### Remove very small events

In [18]:
print(len(all_events_df_5mins))
all_events_df_5mins = all_events_df_5mins[all_events_df_5mins['total_precip']>4].copy()
print(len(all_events_df_5mins))

print(len(all_events_df_10mins))
all_events_df_10mins = all_events_df_10mins[all_events_df_10mins['total_precip']>4].copy()
print(len(all_events_df_10mins))

print(len(all_events_df_30mins))
all_events_df_30mins = all_events_df_30mins[all_events_df_30mins['total_precip']>4].copy()
print(len(all_events_df_30mins))

print(len(all_events_df_60mins))
all_events_df_60mins = all_events_df_60mins[all_events_df_60mins['total_precip']>4].copy()
print(len(all_events_df_60mins))

364232
233113
364232
233125
364232
233146
364232
233128


# Create version for all resolutions, just raw
### Specify just raw columns to keep

In [19]:
raw_cols = []
for col in all_events_df_5mins.columns:
    if not col.endswith('_DMC_10') and not col.endswith('dblnorm'):
        raw_cols.append(col)

### Also remove 'frac_q1', 'frac_q2', 'frac_q3', 'frac_q4'
Because these bring lots of nans (with current calculation method)  
ni is the same as peak mean ratio

In [20]:
cols_to_del = ['frac_q1', 'frac_q2', 'frac_q3', 'frac_q4', 'BSC', 'ni', 'min_intensity']
more_cols_to_del = ['gauge_num',  'start_time', 'end_time',
#                     'duration' ,'total_precip',  'event_num',
                    'peak_mean_ratio_scaled',
                   'heaviest_half']

raw_cols = [x for x in raw_cols if x not in cols_to_del]  
raw_cols = [x for x in raw_cols if x not in more_cols_to_del]    

In [21]:
all_events_df_5mins_raw = all_events_df_5mins[raw_cols]
all_events_df_10mins_raw = all_events_df_10mins[raw_cols]
all_events_df_30mins_raw = all_events_df_30mins[raw_cols]
all_events_df_60mins_raw = all_events_df_60mins[raw_cols]

In [22]:
print(len(all_events_df_5mins_raw), len(all_events_df_10mins_raw), len(all_events_df_30mins_raw), len(all_events_df_60mins_raw))

bad_indices_all = []
for all_events_df in [all_events_df_5mins_raw, all_events_df_10mins_raw, all_events_df_30mins_raw, all_events_df_60mins_raw]:
    columns_with_nan = all_events_df.columns[all_events_df.isnull().any()].tolist()
    print("Columns with at least one NaN value:")
    print(columns_with_nan)
    # #Find indices of rows with any NaNs in the 60-minute data
    bad_indices = all_events_df[all_events_df.isnull().any(axis=1)].index
    bad_indices_lst = bad_indices.tolist()
    print(len(bad_indices_lst))
    if len(bad_indices) >0:
        bad_indices_all.extend(bad_indices_lst)
    
# Drop rows at these indices from all dataframes
all_events_df_5mins_raw = all_events_df_5mins_raw.drop(index=bad_indices_all)
all_events_df_10mins_raw = all_events_df_10mins_raw.drop(index=bad_indices_all)
all_events_df_30mins_raw = all_events_df_30mins_raw.drop(index=bad_indices_all)
all_events_df_60mins_raw = all_events_df_60mins_raw.drop(index=bad_indices_all)   

print(len(all_events_df_5mins_raw), len(all_events_df_10mins_raw), len(all_events_df_30mins_raw), len(all_events_df_60mins_raw))

233113 233125 233146 233128
Columns with at least one NaN value:
[]
0
Columns with at least one NaN value:
[]
0
Columns with at least one NaN value:
[]
0
Columns with at least one NaN value:
[]
0
233113 233125 233146 233128


### Join all into one dataframe

In [23]:
metric_columns =all_events_df_5mins_raw.columns

In [24]:
all_events_df_5mins_raw = all_events_df_5mins_raw.add_suffix('_5m')
all_events_df_10mins_raw = all_events_df_10mins_raw.add_suffix('_10m')
all_events_df_30mins_raw = all_events_df_30mins_raw.add_suffix('_30m')
all_events_df_60mins_raw = all_events_df_60mins_raw.add_suffix('_60m')

In [25]:
all_events_all_res = pd.concat([all_events_df_5mins_raw, all_events_df_10mins_raw,
                               all_events_df_30mins_raw, all_events_df_60mins_raw], axis=1)

In [57]:
all_events_all_res.to_csv("../NotScaled_AllRes.csv",index=False)

# Create version for 5 minutes, DMC and raw
### Specify columns to keep

In [29]:
cols_to_keep = []
for col in all_events_df_5mins.columns:
    if not col.endswith('dblnorm'):
        cols_to_keep.append(col)

In [30]:
cols_to_del = ['frac_q1', 'frac_q2', 'frac_q3', 'frac_q4', 'BSC', 'ni', 'min_intensity', 'T50']
more_cols_to_del = ['gauge_num', 'start_time', 'end_time', 'duration', 'total_precip', 'event_num', 'peak_mean_ratio_scaled', 'heaviest_half']
all_to_del = cols_to_del + more_cols_to_del

# Remove any column that matches exactly or starts with one of the names (e.g., has suffixes like _DMC_10)
cols_to_keep = [col for col in cols_to_keep if not any(col == base or col.startswith(base + '_') for base in all_to_del)]

# Then filter the DataFrame
all_events_df_5mins_filtered = all_events_df_5mins[cols_to_keep]

In [33]:
all_events_df_5mins_filtered.to_csv("../NotScaled_RawVsDMC.csv",index=False)